In [1]:
#1.데이터 로드
import numpy as np
import pandas as pd
train = pd.read_csv('data/train.csv')

X_train = train.drop(columns='Survived')
y_train = train['Survived']

X_test = pd.read_csv('data/test.csv')
PassengerId = X_test['PassengerId'].copy()

In [2]:
#2.EDA
import seaborn as sns

def graph(variable):
    survived = train[train['Survived']==1][variable].value_counts()
    dead = train[train['Survived']==0][variable].value_counts()
    result = pd.DataFrame([survived,dead])
    result.index = ['Survived','Dead']
    result.T.plot(kind='bar', color=['skyblue','gray'], rot=0, figsize=(8,6))

print(train.info())
#graph('Parch')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
#3.데이터 정제
#print(X_train.info())
#print(X_train['Fare'].unique())
#print(X_train.select_dtypes('object').nunique())

#불필요한 컬럼 제거
X_train = X_train.drop(columns=['PassengerId','Name','Ticket','Cabin'])
X_test = X_test.drop(columns=['PassengerId','Name','Ticket','Cabin'])

#SibSp, Parch 컬럼 통합
X_train['family'] = X_train['SibSp']+X_train['Parch']
X_train = X_train.drop(columns=['SibSp','Parch'])

X_test['family'] = X_test['SibSp']+X_test['Parch']
X_test = X_test.drop(columns=['SibSp','Parch'])

In [4]:
#4.결측치 및 이상치 처리
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_test['Age'] = X_test['Age'].fillna(X_train['Age'].mean())

X_train['Embarked'] = X_train['Embarked'].fillna(X_train['Embarked'].value_counts().idxmax())
X_test['Embarked'] = X_test['Embarked'].fillna(X_train['Embarked'].value_counts().idxmax())

X_train['Fare'] = X_train['Fare'].fillna(X_train['Fare'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_train['Fare'].mean())

In [5]:
# #BK_4.Age컬럼 결측치 대체법
# #범주형은 인코딩하고, corr()을 이용해서 상관계수 높은 컬럼 추출

# #print(X_train.info())
# #print(X_train.select_dtypes('object').nunique())

# #X_train = pd.get_dummies(X_train,columns=['Sex','Embarked'])

# #X_train[['Age','Fare','Parch','SibSp','Pclass','Sex_female','Sex_male','Embarked_C','Embarked_Q','Embarked_S','Family']].corr()['Age']

# #그 컬럼별 group by해서 age 평균을 각 행의 결측치에 대체

# Age_By_train = pd.DataFrame(X_train.groupby(['Pclass','family'])['Age'].mean())
# Age_By_train = Age_By_train.reset_index()

# Age_By_test = pd.DataFrame(X_test.groupby(['Pclass','family'])['Age'].mean())
# Age_By_test = Age_By_test.reset_index()


# for index, row in Age_By_Pclass.iterrows():
#     X_train.loc[(X_train['Pclass'] == row['Pclass']) & (X_train['family'] == row['family']) & (X_train['Age'].isna()), 'Age'] = row['Age']
#     X_test.loc[(X_test['Pclass'] == row['Pclass']) & (X_test['family'] == row['family']) & (X_test['Age'].isna()), 'Age'] = row['Age']

# # import seaborn as sns
# # import matplotlib.pyplot as plt

# # fig,ax = plt.subplots(2,figsize=(40,30))
# # sns.countplot(data=X_train,x='Age',ax=ax[0])
# # sns.countplot(data=X_test,x='Age',ax=ax[1])
# # plt.show()

In [6]:
#5.인코딩
X_train = pd.get_dummies(X_train,columns=['Sex','Embarked','Pclass'])
X_test = pd.get_dummies(X_test,columns=['Sex','Embarked','Pclass'])

X_train_enc = X_train.select_dtypes('bool').astype('int')
X_test_enc = X_test.select_dtypes('bool').astype('int')

In [7]:
#6.스케일링
from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(exclude='bool').copy()
X_test_num = X_test.select_dtypes(exclude='bool').copy()

scale = StandardScaler().fit(X_train_num)

X_train_std = scale.transform(X_train_num)
X_test_std = scale.transform(X_test_num)

X_train_std = pd.DataFrame(X_train_std,columns=['Age','family','Fare'])
X_test_std = pd.DataFrame(X_test_std,columns=['Age','family','Fare'])

X_train = pd.concat([X_train_std,X_train_enc],axis=1)
X_test = pd.concat([X_test_std,X_test_enc],axis=1)

In [8]:
#7.특성 선택
#RFE
from sklearn.feature_selection import RFE, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

X = X_train.drop(columns=['Sex_male','Embarked_C','Pclass_1'])
y = y_train
test = X_test.drop(columns=['Sex_male','Embarked_C','Pclass_1'])

model = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)

rfe = RFE(model, n_features_to_select=6)  # 선택할 특성의 개수를 지정합니다.

all_columns = X.columns

# RFE 알고리즘을 사용하여 특성 선택을 수행합니다.
rfe_fit = rfe.fit(X, y)

# 선택된 특성들을 출력합니다.
selected_mask = rfe_fit.support_
print(all_columns[selected_mask])
print(rfe_fit.estimator_.feature_importances_)

X = X_train[['Age', 'family', 'Fare', 'Sex_female', 'Embarked_S', 'Pclass_3']]
test = X_test[['Age', 'family', 'Fare', 'Sex_female', 'Embarked_S', 'Pclass_3']]



#SelectKBest을 활용한 특성 선택 탐색
# from sklearn.feature_selection import SelectKBest, f_classif

# X = X_train.drop(columns=['Sex_male','Embarked_C','Pclass_1'])
# y = y_train
# test = X_test.drop(columns=['Sex_male','Embarked_C','Pclass_1'])

# all_columns = X.columns

# sel = SelectKBest(score_func=f_classif,k=6)
# sel_fit = sel.fit(X,y)
# X = sel_fit.transform(X)
# test = sel_fit.transform(test)

# selected_mask = sel_fit.get_support()
# print(all_columns[selected_mask])
# print(X)
# print(test)

# print(all_columns)
# print(sel_fit.scores_.astype('int'))

# X = X_train[['Age','family','Sex_female','Embarked_S','Pclass_2','Pclass_3']]
# y = y_train
# test = X_test[['Age','family','Sex_female','Embarked_S','Pclass_2','Pclass_3']]



#statsmodels.Logit() 통계적 분석 결과 표(p-value, coef 값)를 통한 특성 선택 
# import statsmodels.api as sm

# X = X_train.drop(columns=['Sex_male','Embarked_C','Pclass_1'])
# y = y_train
# test = X_test.drop(columns=['Sex_male','Embarked_C','Pclass_1'])

# model = sm.Logit(y,X)
# model_fit = model.fit()
# print(model_fit.summary())

# X = X_train[['Age','Fare','Sex_female','Embarked_S','Pclass_2','Pclass_3']]
# y = y_train
# test = X_test[['Age','Fare','Sex_female','Embarked_S','Pclass_2','Pclass_3']]

Index(['Age', 'family', 'Fare', 'Sex_female', 'Embarked_S', 'Pclass_3'], dtype='object')
[0.16663802 0.21722868 0.08560329 0.39942311 0.02429701 0.10680989]


In [9]:
#8.하이퍼파라미터 최적화
from skopt import BayesSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# 모델 정의
models = {
    'LogisticRegression': LogisticRegression(),
    'RandomForestClassifier': RandomForestClassifier(),
    'XGBClassifier': XGBClassifier(),
    'LGBMClassifier': LGBMClassifier(verbose=-1)
    # 'CatBoostClassifier': CatBoostClassifier()
}

# 하이퍼파라미터 탐색 범위 지정
search_space = {
    'LogisticRegression': {
        'max_iter': (100, 1000),
        'C': (0.01, 100)
    },
    'RandomForestClassifier': {
        'n_estimators': (10, 1000),   # 트리 개수
        'max_depth': (3, 20),         # 트리 최대 깊이
        'min_samples_split': (2, 10), # 분할을 위한 최소 샘플 수
        'min_samples_leaf': (1, 10)   # 잎 노드에 필요한 최소 샘플 수
    },
    'XGBClassifier': {
        'n_estimators': (10, 1000),
        'max_depth': (3, 20),
        'min_child_weight': (1, 20),
        'learning_rate': (0.001, 1.0),
        'subsample': (0.1,1)
    },
    'LGBMClassifier': {
        'n_estimators': (10, 1000),
        'max_depth': (3, 20),
        'min_child_weight': (1, 20),
        'learning_rate': (0.001, 1.0),
        'num_leaves': (10,1000)
    }
    # 'CatBoostClassifier': {
    #     'iterations': (10, 1000),
    #     'depth': (3, 20),
    #     'min_child_samples': (1, 20),
    #     'learning_rate': (0.001, 1.0)
    #     'l2_leaf_reg': (1,10)
    # }
}

# 베이지안 최적화를 사용한 자동화된 하이퍼파라미터 튜닝
best_params = {}
for model_name, model in models.items():
    opt = BayesSearchCV(
        model,
        search_space[model_name],
        n_iter=10,   # 반복 횟수
        cv=5,       # 교차 검증 폴드 수
        scoring='accuracy'
    )
    opt.fit(X, y)
    best_params[model_name] = opt.best_params_

# 최적 하이퍼파라미터 출력
for model_name, params in best_params.items():
    print(model_name, "best parameter:", params)

LogisticRegression best parameter: OrderedDict([('C', 45.95039732036445), ('max_iter', 645)])
RandomForestClassifier best parameter: OrderedDict([('max_depth', 16), ('min_samples_leaf', 4), ('min_samples_split', 9), ('n_estimators', 93)])
XGBClassifier best parameter: OrderedDict([('learning_rate', 0.690570437583915), ('max_depth', 9), ('min_child_weight', 17), ('n_estimators', 356), ('subsample', 0.7242771491724633)])
LGBMClassifier best parameter: OrderedDict([('learning_rate', 0.24148357781556884), ('max_depth', 12), ('min_child_weight', 16), ('n_estimators', 670), ('num_leaves', 630)])


In [10]:
#9.모델 생성

#LogisticRegression
model_lr = LogisticRegression(
                    max_iter=909,
                    C=59.320,
                    random_state=999)
# model_lr.fit(X,y)
# y_predict_lr = model_lr.predict(test)

#RandomForestClassifier
model_rf = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)
# model_rf.fit(X,y)
# y_predict_rf = model_rf.predict(test)

#XGBClassifier
model_xgb = XGBClassifier(
                    max_depth=5,
                    n_estimators=461,
                    min_child_weight=11,
                    learning_rate=0.292,
                    subsample=0.613,
                    random_state=999)
# model_xgb.fit(X,y)
# y_predict_xgb = model_xgb.predict(test)

#LGBMClassifier
model_lgbm = LGBMClassifier(
                    max_depth=11,
                    n_estimators=52,
                    min_child_weight=12,
                    learning_rate=0.297,
                    num_leaves=407,
                    verbose=-1,
                    random_state=999)
# model_lgbm.fit(X,y)
# y_predict_lgbm = model_lgbm.predict(test)

#CatBoostClassifier
# model_cat = CatBoostClassifier(
#                     depth=2,
#                     iterations=100,
#                     learning_rate=0.2, 
#                     loss_function='MultiClass')
# model_cat.fit(X,y,verbose=False)
# y_predict_cat = model_cat.predict(test)
# y_predict_cat = np.array(y_predict_cat).flatten() #array in array 형태를 array로 변환

In [11]:
#10.모델 학습 및 평가

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score

kfold = KFold(n_splits=5,shuffle=True,random_state=999)
print(cross_validate(model_lr, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
print(cross_validate(model_rf, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
print(cross_validate(model_xgb, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
print(cross_validate(model_lgbm, X, y, cv=kfold, scoring='accuracy')['test_score'].mean().round(3))
# print(cross_validate(model_cat, X, y, cv=kfold, scoring='accuracy',fit_params={"verbose":False})['test_score'].mean().round(3))

# 필요시 데이터 분리하기
# from sklearn.model_selection import train_test_split
# X1, X2, y1, y2 = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=999)

# print(accuracy_score(y2,y_predict_lr).round(3))
# print(f1_score(y2,y_predict_lr).round(3))
# probas_lr = model_lr.predict_proba(X2)
# print(roc_auc_score(y2,probas_lr[:,1]).round(3))

0.808
0.825
0.818
0.835


In [12]:
#11.실 데이터 예측 및 제출
model_rf = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)
model_rf.fit(X,y)
y_predict_rf = model_rf.predict(test)

'''
#Best Parameter
model_rf = RandomForestClassifier(
                    max_depth=7,
                    n_estimators=200,
                    max_features='sqrt',
                    min_samples_split=2,
                    min_samples_leaf=1,
                    random_state=999)
model_rf.fit(X,y)
y_predict_rf = model_rf.predict(test)
'''

obj = {'PassengerId':PassengerId,'Survived':y_predict_rf}
result = pd.DataFrame(obj)
result.to_csv('result/result.csv', index = False)